**For this notebook to work, the prompt command code needs to be "pyspark" instead of "jupyter notebook"**

**Documentation:**

* PySpark RDD: https://spark.apache.org/docs/2.2.0/api/python/pyspark.html#pyspark.RDD

* PySpark SQL: https://spark.apache.org/docs/latest/sql-programming-guide.html

* PySpark SQL DataFrame: https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [1]:
# Import pyspark.sql modules
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.types import FloatType

# Packages to upload to firebase
import json
import requests

In [2]:
# URL for the Firebase database
database_url = 'https://neighborhood-score-la.firebaseio.com/.json'

In [3]:
# Clean Firebase database
requests.put(database_url, data=json.dumps(None))

<Response [200]>

In [4]:
# Initiate the sqlSession, sqlContext and get the link for the Spark UI
sqlSession = SparkSession.builder.master("local").appName("Neighborhood-Scorer").getOrCreate()
sqlContext = SQLContext(sc)
sqlContext.sparkSession

### Load the Crime Dataset to PySpark and MapReduce to Aggregate Data

In [5]:
# Read the csv with neighborhood crime data
#df1 = sqlContext.read.csv('Crime_Data_2019_Neighborhoods_v5.csv', header=True)
df1 = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://3.86.145.6/dsci551").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "CRIME").option("user", "matt").option("password", "Sup3rDup3rC@li").load()
df1.show()

+---+---------+----------+--------------------+-------------------+
| id|    dr_no|  date_occ|        neighborhood|crime_weighted_norm|
+---+---------+----------+--------------------+-------------------+
|  1|191907191|2019-03-07|              Sylmar|  0.113706949000000|
|  2|190125334|2019-10-16|Downtown Los Angeles|  0.246650906000000|
|  3|191920961|2019-12-21|    North Hills East|  0.113706949000000|
|  4|190604395|2019-01-08|          Central LA|  0.113706949000000|
|  5|191310615|2019-04-29|   South Los Angeles|  0.113706949000000|
|  6|191419522|2019-09-02|         Westchester|  0.113706949000000|
|  7|190123550|2019-09-20|Downtown Los Angeles|  0.246650906000000|
|  8|191915118|2019-08-17|              Sylmar|  0.113706949000000|
|  9|190811511|2019-06-19|           Brentwood|  0.113706949000000|
| 10|191605013|2019-01-28|          Sun Valley|  0.113706949000000|
| 11|190129579|2019-12-17|Downtown Los Angeles|  0.113706949000000|
| 12|191116816|2019-09-17|Northeast Los Ang...| 

In [6]:
# MapReduce to get the count of crimes per neighborhood
df2 = df1.groupBy('Neighborhood').count()
df2 = df2.withColumnRenamed('count', 'CrimeCount')
df2.show()

+--------------------+----------+
|        Neighborhood|CrimeCount|
+--------------------+----------+
|           Mar Vista|      1364|
|          West Hills|       793|
|           Hollywood|       165|
|       Glassell Park|       232|
|    Pico - Robertson|       844|
|              Harbor|       902|
|            Mid City|      8345|
|Downtown Los Angeles|     13409|
|              Reseda|      2511|
| Crescenta Highlands|         4|
|    North of Montana|         2|
|          Central LA|     24753|
|          McLaughlin|       231|
|       Culver - West|       303|
|       Baldwin Hills|       142|
|      Elysian Valley|        67|
|          Eagle Rock|        10|
|        Sunset Strip|        14|
|        North Arroyo|         1|
|        Century City|       465|
+--------------------+----------+
only showing top 20 rows



In [7]:
# MapReduce to the the average crime score per neighborhood
df3 = df1.groupBy('Neighborhood').agg({'Crime_Weighted_Norm': 'avg'})
df3 = df3.withColumnRenamed('avg(Crime_Weighted_Norm)', 'CrimeScore')
df3.show()

+--------------------+--------------------+
|        Neighborhood|          CrimeScore|
+--------------------+--------------------+
|           Mar Vista|0.165546059546921...|
|          West Hills|0.157788162368222...|
|           Hollywood|0.170648896006061...|
|       Glassell Park|0.130160961318966...|
|    Pico - Robertson|0.160393906523697...|
|              Harbor|0.163150450230599...|
|            Mid City|0.159816786333373...|
|Downtown Los Angeles|0.169588955401074...|
|              Reseda|0.149907676853843...|
| Crescenta Highlands|0.195663656500000...|
|    North of Montana|0.022944958000000...|
|          Central LA|0.165907768608290...|
|          McLaughlin|0.132804030666667...|
|       Culver - West|0.159975528287129...|
|       Baldwin Hills|0.158982026387324...|
|      Elysian Valley|0.220146479298507...|
|          Eagle Rock|0.165961695300000...|
|        Sunset Strip|0.188067239071429...|
|        North Arroyo|0.037952846000000...|
|        Century City|0.17109346

### Load the House Dataset to PySpark and MapReduce to Aggregate Data

In [8]:
# Read the csv with neighborhood housing data
#df4 = sqlContext.read.csv('neighborhood_housing_score.csv', header=True)
df4 = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://3.86.145.6/dsci551").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "rent_scores").option("user", "matt").option("password", "Sup3rDup3rC@li").load()
df4 = df4.withColumnRenamed('RegionName', 'Neighborhood')
df4.show()

+---+--------------------+---------+--------------------+
| id|        neighborhood|size_rank|       housing_score|
+---+--------------------+---------+--------------------+
|  1|   South Los Angeles|        4|442013.5735294117...|
|  2|Southeast Los Ang...|        8|367733.6911764706...|
|  3|           Hollywood|       18|847758.7058823529...|
|  4|            Mid City|       62|986030.0147058824...|
|  5|            Van Nuys|       64|535259.6764705882...|
|  6|          Sun Valley|       68|513386.4852941176...|
|  7|          Northridge|       69|670616.4705882353...|
|  8|              Sylmar|       74|483452.1617647059...|
|  9|       Boyle Heights|       75|424736.1617647059...|
| 10|     North Hollywood|       78|613276.5441176471...|
| 11|           San Pedro|       79|583256.8382352941...|
| 12|           Koreatown|       82|629124.5441176471...|
| 13|      Woodland Hills|       83|780429.2647058824...|
| 14|        Sherman Oaks|       85|951315.7647058824...|
| 15|         

In [9]:
# MapReduce to the the average size rank per neighborhood
df5 = df4.groupBy('Neighborhood').agg({'size_rank': 'avg'})
df5 = df5.withColumnRenamed('avg(size_rank)', 'SizeRank')
df5.show()

+----------------+--------+
|    Neighborhood|SizeRank|
+----------------+--------+
|       Mar Vista|   238.0|
| Harvard Heights|   413.0|
|      West Hills|   288.0|
|       Hollywood|    18.0|
|   Glassell Park|   540.0|
|        Mid City|    62.0|
|          Reseda|   100.0|
|         Del Rey|   331.0|
|   Playa Del Rey|   886.0|
|         Sunland|   866.0|
|    Shadow Hills|  4441.0|
|  Elysian Valley|  2819.0|
|   Mid City West|   208.0|
|      Eagle Rock|   382.0|
|    Elysian Park| 11212.0|
|    Century City|  3269.0|
|      Pico-Union|   770.0|
|       Brentwood|   360.0|
|   Cheviot Hills|   988.0|
|West Los Angeles|  1179.0|
+----------------+--------+
only showing top 20 rows



In [10]:
# MapReduce to the the average size rank per neighborhood
df6 = df4.groupBy('Neighborhood').agg({'Housing_Score': 'avg'})
df6 = df6.withColumnRenamed('avg(Housing_Score)', 'HousingScore')
df6.show()

+----------------+--------------------+
|    Neighborhood|        HousingScore|
+----------------+--------------------+
|       Mar Vista|1330032.941176470...|
| Harvard Heights|681784.0735294118...|
|      West Hills|662252.3382352941...|
|       Hollywood|847758.7058823529...|
|   Glassell Park|735981.5588235294...|
|        Mid City|986030.0147058824...|
|          Reseda|501339.0441176470...|
|         Del Rey|1011292.397058823...|
|   Playa Del Rey|972849.4558823529...|
|         Sunland|556602.8088235294...|
|    Shadow Hills|715190.3529411765...|
|  Elysian Valley|687034.6764705882...|
|   Mid City West|1532142.411764705...|
|      Eagle Rock|796614.2500000000...|
|    Elysian Park|590152.0735294118...|
|    Century City|1099622.073529411...|
|      Pico-Union|549041.7647058824...|
|       Brentwood|2223105.058823529...|
|   Cheviot Hills|1664359.411764705...|
|West Los Angeles|1257659.455882353...|
+----------------+--------------------+
only showing top 20 rows



### Load the School Dataset to PySpark and MapReduce to Aggregate Data

In [11]:
# Read the csv with neighborhood school data
#df7 = sqlContext.read.csv('ACT19_LA_neighborhood_socre.csv', header=True)
df7 = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://3.86.145.6/dsci551").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "ACT_neighborhoods").option("user", "matt").option("password", "Sup3rDup3rC@li").load()
df7 = df7.drop('id')
df7 = df7.withColumnRenamed('RegionName', 'Neighborhood')
df7 = df7.withColumnRenamed('averages', 'AvgACT')
df7 = df7.withColumn("AVG_SCR_READ", df7["AVG_SCR_READ"].cast(FloatType()))
df7 = df7.withColumn("AVG_SCR_ENG", df7["AVG_SCR_ENG"].cast(FloatType()))
df7 = df7.withColumn("AVG_SCR_MATH", df7["AVG_SCR_MATH"].cast(FloatType()))
df7 = df7.withColumn("AVG_SCR_SCI", df7["AVG_SCR_SCI"].cast(FloatType()))
df7 = df7.withColumn("AvgACT", df7["AvgACT"].cast(FloatType()))
df7.show()

+--------------+---------+------------+-----------+------------+-----------+
|  neighborhood|   AvgACT|AVG_SCR_READ|AVG_SCR_ENG|AVG_SCR_MATH|AVG_SCR_SCI|
+--------------+---------+------------+-----------+------------+-----------+
|  Agoura Hills|     25.5|        27.0|       26.0|        25.0|       24.0|
|      Alhambra|   25.375|        25.5|       26.0|        25.5|       24.5|
|     Allentown|     23.5|        25.0|       24.0|        22.0|       23.0|
|       Arcadia|     28.0|        28.0|       29.0|        28.0|       27.0|
|         Azusa|     17.5|        17.0|       16.0|        18.0|       19.0|
|  Baldwin Park|     20.0|        21.0|       19.0|        20.0|       20.0|
|       Bassett|    22.25|        23.0|       22.0|        22.0|       22.0|
|          Bell|    17.75|        18.0|       17.0|        18.0|       18.0|
| Beverly Hills|     27.5|        28.0|       29.0|        27.0|       26.0|
|       Burbank|   24.875|        25.5|       25.5|        24.5|       24.0|

### Merge All Datasets Indexing by Neighborhood

In [12]:
# Join all dataframes by neighborhood
df_merged = df2.join(df3, 'Neighborhood', how='fullouter').join(df5, 'Neighborhood', how='fullouter').join(df6, 'Neighborhood', how='fullouter').join(df7, 'Neighborhood', how='fullouter')
df_merged.show()

+--------------------+----------+--------------------+--------+--------------------+------+------------+-----------+------------+-----------+
|        Neighborhood|CrimeCount|          CrimeScore|SizeRank|        HousingScore|AvgACT|AVG_SCR_READ|AVG_SCR_ENG|AVG_SCR_MATH|AVG_SCR_SCI|
+--------------------+----------+--------------------+--------+--------------------+------+------------+-----------+------------+-----------+
|     Harvard Heights|      null|                null|   413.0|681784.0735294118...|  null|        null|       null|        null|       null|
|           Mar Vista|      1364|0.165546059546921...|   238.0|1330032.941176470...|  null|        null|       null|        null|       null|
|          West Hills|       793|0.157788162368222...|   288.0|662252.3382352941...|  null|        null|       null|        null|       null|
|       Glassell Park|       232|0.130160961318966...|   540.0|735981.5588235294...|  null|        null|       null|        null|       null|
|     

In [13]:
print(f'There are {df_merged.count()} neighborhoods in the dataset')

There are 224 neighborhoods in the dataset


### Keep Only Neighborhoods from the City of Los Angeles

Source of the list of Los Angeles City neighborhoods: http://www.laalmanac.com/cities/ci93.php

In [14]:
LACity_Neighborhoods = ["Del Rey",
"L.A.International Airport",
"Mar Vista",
"Marina Peninsula",
"Playa Del Rey",
"Playa Vista",
"Venice",
"Westchester",
"La Tuna Canyon",
"Lakeview Terrace",
"Shadow Hills",
"Sunland",
"Tujunga",
"Arlington Heights",
"Boyle Heights",
"Brookside",
"Byzantine-Latino Quarter",
"Cahuega Pass",
"Carthay",
"Chinatown",
"Central City East",
"Country Club Park",
"Downtown LA",
"East Hollywood",
"Echo Park",
"Elysian Park",
"Fremont Place",
"Griffith Park",
"Hancock Park",
"Harvard Heights",
"Historic Filipinotown",
"Historic West Adams",
"Hollywood",
"Koreatown",
"La Brea Hancock",
"Larchmont Village",
"Los Feliz",
"Maplewood-St. Andrews",
"Melrose",
"Mid-City",
"Mid-City West",
"Miracle Mile",
"Olympic Park",
"Picfair Village",
"Pico Park",
"Pico Union",
"Rampart Village",
"Redondo Sycamore",
"St. Andrews Square",
"Silver Lake",
"South Carthay",
"Sycamore Square",
"Western Heights",
"Western Wilton",
"Westlake",
"Wilshire Center",
"Wilshire Park",
"Wilshire Vista",
"Wilshire Vista Heights",
"Wilshire Square",
"Windsor Village",
"Harbor City",
"Harbor Gateway",
"San Pedro",
"Terminal Island",
"Wilmington",
"Atwater Village",
"Boyle Heights",
"Cypress Park",
"Eagle Rock",
"El Sereno",
"Elysian Valley",
"Glassell Park",
"Hermon",
"Highland Park",
"Lincoln Heights",
"Montecito Heights",
"Monterey Hills",
"Mt. Washington",
"Sycamore Grove",
"University Hills",
"Arleta",
"Canoga Park",
"Chatsworth",
"Encino",
"Granada Hills",
"Lake Balboa",
"Mission Hills",
"North Hills",
"North Hollywood",
"Northridge",
"Pacoima",
"Panorama City",
"Porter Ranch",
"Reseda",
"Sherman Oaks",
"Studio City",
"Sun Valley",
"Sylmar",
"Tarzana",
"Toluca Lake",
"Toluca Terrace",
"Toluca Woods",
"Valley Glen",
"Valley Village",
"Van Nuys",
"West Hills",
"West Toluca Lake",
"Winnetka",
"Woodland Hills",
"Arlington Park",
"Baldwin Hills",
"Berkeley Square",
"Crenshaw",
"Crenshaw Manor",
"Historic West Adams",
"Hyde Park",
"Jefferson Park",
"Kinney Heights",
"Leimert Park",
"South Los Angeles",
"Southeast Los Angeles",
"University Park",
"Watts",
"West Adams",
"West Adams Terrace",
"Bel Air",
"Beverly Crest",
"Beverly Glen",
"Brentwood",
"Century City",
"Cheviot Hills",
"Crestview",
"Pacific Palisades",
"Palms",
"Rancho Park",
"South Carthay",
"South Robertson",
"Topanga State Park",
"West Los Angeles",
"Westside Village",
"Westwood"]

In [15]:
df_merged_LACity = df_merged.where(df_merged.Neighborhood.isin(LACity_Neighborhoods))
df_merged_LACity.show()

+----------------+----------+--------------------+--------+--------------------+------+------------+-----------+------------+-----------+
|    Neighborhood|CrimeCount|          CrimeScore|SizeRank|        HousingScore|AvgACT|AVG_SCR_READ|AVG_SCR_ENG|AVG_SCR_MATH|AVG_SCR_SCI|
+----------------+----------+--------------------+--------+--------------------+------+------------+-----------+------------+-----------+
| Harvard Heights|      null|                null|   413.0|681784.0735294118...|  null|        null|       null|        null|       null|
|       Mar Vista|      1364|0.165546059546921...|   238.0|1330032.941176470...|  null|        null|       null|        null|       null|
|      West Hills|       793|0.157788162368222...|   288.0|662252.3382352941...|  null|        null|       null|        null|       null|
|   Glassell Park|       232|0.130160961318966...|   540.0|735981.5588235294...|  null|        null|       null|        null|       null|
|       Hollywood|       165|0.170

In [16]:
print(f'There are {df_merged_LACity.count()} neighborhoods in the dataset')

There are 92 neighborhoods in the dataset


### MapReduce the PySpark DataFrame into a Dictionary

This will be used as the data source for the creation of the jsons to be uploaded

In [17]:
# Convert from PySpark dataframe to json
json_data = df_merged_LACity.toJSON().collect()

In [18]:
# Read the json creating elements by neighborhood
list_data = [json.loads(json_data[i]) for i in range(len(json_data))]

In [19]:
# Extract the names of all neighborhoods and the features created
all_neighborhoods = [x.Neighborhood for x in df_merged_LACity.select('Neighborhood').distinct().collect()]
all_features = df_merged_LACity.columns[1:]

In [20]:
# First create a dict with index 1 being neighborhood and index 2 being each feature, with the values as None
# This is necessary to ensure all Neihgborhoods have all keys, which prevents problems down the line
dict_by_neighborhood = {}
for neigh in all_neighborhoods:
    dict_by_neighborhood[neigh] = {all_features[0]: None,
                                   all_features[1]: None,
                                   all_features[2]: None,
                                   all_features[3]: None,
                                   all_features[4]: None,
                                   all_features[5]: None,
                                   all_features[6]: None,
                                   all_features[7]: None,
                                   all_features[8]: None}        

In [21]:
# Now fill each [Neighborhood][Feature] with the values existing in the data
# Missing values will remain as 'None'
for i in range(len(list_data)):
    for feat in all_features:      
        try:
            dict_by_neighborhood[list_data[i]['Neighborhood']][feat] = list_data[i][feat]
        except:
            pass

In [22]:
dict_by_neighborhood

{'Harvard Heights': {'CrimeCount': None,
  'CrimeScore': None,
  'SizeRank': 413.0,
  'HousingScore': 681784.0735294118,
  'AvgACT': None,
  'AVG_SCR_READ': None,
  'AVG_SCR_ENG': None,
  'AVG_SCR_MATH': None,
  'AVG_SCR_SCI': None},
 'Mar Vista': {'CrimeCount': 1364,
  'CrimeScore': 0.165546059546921,
  'SizeRank': 238.0,
  'HousingScore': 1330032.9411764706,
  'AvgACT': None,
  'AVG_SCR_READ': None,
  'AVG_SCR_ENG': None,
  'AVG_SCR_MATH': None,
  'AVG_SCR_SCI': None},
 'West Hills': {'CrimeCount': 793,
  'CrimeScore': 0.157788162368222,
  'SizeRank': 288.0,
  'HousingScore': 662252.3382352941,
  'AvgACT': None,
  'AVG_SCR_READ': None,
  'AVG_SCR_ENG': None,
  'AVG_SCR_MATH': None,
  'AVG_SCR_SCI': None},
 'Glassell Park': {'CrimeCount': 232,
  'CrimeScore': 0.130160961318966,
  'SizeRank': 540.0,
  'HousingScore': 735981.5588235294,
  'AvgACT': None,
  'AVG_SCR_READ': None,
  'AVG_SCR_ENG': None,
  'AVG_SCR_MATH': None,
  'AVG_SCR_SCI': None},
 'Hollywood': {'CrimeCount': 165,
  'Cr

'# Convert the PySpark Distributed Dataframe to a list of dictionaries to the uploaded to firebase
dict_by_neighborhood = df_merged.rdd.map(lambda x: {x['Neighborhood']: {'CrimeCount': x['CrimeCount'], 
                                                                        'CrimeScore': x['CrimeScore'],
                                                                        'SizeRank': x['SizeRank'],
                                                                        'HousingScore': x['HousingScore'],
                                                                        'AvgScrRead': x['AvgScrRead'],
                                                                        'AvgScrEng': x['AvgScrEng'],
                                                                        'AvgScrMath': x['AvgScrMath'],
                                                                        'AvgScrSci': x['AvgScrSci'],
                                                                        'AvgACT': x['AvgACT']}}).collect()

### Create the Dictionaries to be Uploaded to Firebase

In [23]:
# Create dataset with all data for each neighborhood 
final_neighborhood = {'NeighborhoodData': dict_by_neighborhood}

In [24]:
# Create a dictionary with CrimeCount per Neighborhood
CrimeCount_dict = {i:dict_by_neighborhood[i]['CrimeCount'] for i in dict_by_neighborhood}
final_CrimeCount = {'CrimeCount': CrimeCount_dict}

In [25]:
# Create a dictionary with CrimeScore per Neighborhood
CrimeScore_dict = {i:dict_by_neighborhood[i]['CrimeScore'] for i in dict_by_neighborhood}
final_CrimeScore = {'CrimeScore': CrimeScore_dict}

In [26]:
# Create a dictionary with SizeRank per Neighborhood
SizeRank_dict = {i:dict_by_neighborhood[i]['SizeRank'] for i in dict_by_neighborhood}
final_SizeRank = {'SizeRank': SizeRank_dict}

In [27]:
# Create a dictionary with HousingScore per Neighborhood
HousingScore_dict = {i:dict_by_neighborhood[i]['HousingScore'] for i in dict_by_neighborhood}
final_HousingScore = {'HousingScore': HousingScore_dict}

In [28]:
# Create a dictionary with AvgScrRead per Neighborhood
AvgScrRead_dict = {i:dict_by_neighborhood[i]['AVG_SCR_READ'] for i in dict_by_neighborhood}
final_AvgScrRead = {'AvgScrRead': AvgScrRead_dict}

In [29]:
# Create a dictionary with AvgScrEng per Neighborhood
AvgScrEng_dict = {i:dict_by_neighborhood[i]['AVG_SCR_ENG'] for i in dict_by_neighborhood}
final_AvgScrEng = {'AvgScrEng': AvgScrEng_dict}

In [30]:
# Create a dictionary with AvgScrMath per Neighborhood
AvgScrMath_dict = {i:dict_by_neighborhood[i]['AVG_SCR_MATH'] for i in dict_by_neighborhood}
final_AvgScrMath = {'AvgScrMath': AvgScrMath_dict}

In [31]:
# Create a dictionary with AvgScrSci per Neighborhood
AvgScrSci_dict = {i:dict_by_neighborhood[i]['AVG_SCR_SCI'] for i in dict_by_neighborhood}
final_AvgScrSci = {'AvgScrSci': AvgScrSci_dict}

In [32]:
# Create a dictionary with AvgACT per Neighborhood
AvgACT_dict = {i:dict_by_neighborhood[i]['AvgACT'] for i in dict_by_neighborhood}
final_AvgACT = {'AvgACT': AvgACT_dict}

### Upload Data to Firebase

In [33]:
# Patch the Neighborhood Data
patch_neighborhood = requests.patch(database_url, data=json.dumps(final_neighborhood))
print(f'Patching Neighborhood Data: {patch_neighborhood.reason}')

Patching Neighborhood Data: OK


In [34]:
# Patch the CrimeCount Dictionary
patch_crimecount = requests.patch(database_url, data=json.dumps(final_CrimeCount))
print(f'Patching CrimeCount Data: {patch_crimecount.reason}')

Patching CrimeCount Data: OK


In [35]:
# Patch the CrimeScore Dictionary
patch_crimescore = requests.patch(database_url, data=json.dumps(final_CrimeScore))
print(f'Patching CrimeScore Data: {patch_crimescore.reason}')

Patching CrimeScore Data: OK


In [36]:
# Patch the SizeRank Data
patch_sizerank = requests.patch(database_url, data=json.dumps(final_SizeRank))
print(f'Patching SizeRank Data: {patch_sizerank.reason}')

Patching SizeRank Data: OK


In [37]:
# Patch the HousingScore Dictionary
patch_housingscore = requests.patch(database_url, data=json.dumps(final_HousingScore))
print(f'Patching HousingScore Data: {patch_housingscore.reason}')

Patching HousingScore Data: OK


In [38]:
# Patch the AvgScrRead Dictionary
patch_avgscrread = requests.patch(database_url, data=json.dumps(final_AvgScrRead))
print(f'Patching AvgScrRead Data: {patch_avgscrread.reason}')

Patching AvgScrRead Data: OK


In [39]:
# Patch the AvgScrEng Dictionary
patch_avgscreng = requests.patch(database_url, data=json.dumps(final_AvgScrEng))
print(f'Patching AvgScrEng Data: {patch_avgscreng.reason}')

Patching AvgScrEng Data: OK


In [40]:
# Patch the AvgScrMath Dictionary
patch_avgscrmath = requests.patch(database_url, data=json.dumps(final_AvgScrMath))
print(f'Patching AvgScrMath Data: {patch_avgscrmath.reason}')

Patching AvgScrMath Data: OK


In [41]:
# Patch the AvgScrSci Dictionary
patch_avgscrsci = requests.patch(database_url, data=json.dumps(final_AvgScrSci))
print(f'Patching AvgScrSci Data: {patch_avgscrsci.reason}')

Patching AvgScrSci Data: OK


In [42]:
# Patch the AvgACT Dictionary
patch_avgact = requests.patch(database_url, data=json.dumps(final_AvgACT))
print(f'Patching AvgACT Data: {patch_avgact.reason}')

Patching AvgACT Data: OK
